In [ ]:
%matplotlib inline
#%pylab
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mpc
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

import scipy
from scipy import interpolate
import pandas as pd
import itertools
from root_numpy import root2array, root2rec, tree2rec, array2root
#from ROOT import TChain

In [ ]:
def get_data_frame(tfile,ttree):
    rec = root2array(tfile,ttree)
    return pd.DataFrame(rec)

print 'Reading in real data...'
REALDATA = get_data_frame("/Users/davidkaleko/larlite/UserDev/MichelReco/MichelCluster/mac/michel_clusters_realdata.root","out_tree")


In [ ]:
print "\t!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!"
print "\t!!!!~~~~~~~~~~~~~~NO CUTS~~~~~~~~~~~~~!!!!"
print "\t!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!"
#kaleko: this used to be _largest_cluster_charge... cosmic_ana doesn't make that variable
S = REALDATA.query    ('_michel_clustered_charge > 0')

plt.figure(figsize=(16,24))

plt.subplot(3,2,1)
plt.grid(True)
plt.hist(np.array(S['_lowest_chi']),bins=100,range=(0,1),color='b',alpha=0.5,label="RealData")
plt.tick_params(labelsize=20)
plt.xlabel('Lowest Chi',fontsize=20)
plt.ylabel('Count',fontsize=20)
plt.legend(fontsize=20)

plt.subplot(3,2,2)
plt.grid(True)
plt.hist(np.array(S['_mean_chi']),bins=100,range=(0,1),color='b',alpha=0.5,label="RealData")
plt.tick_params(labelsize=20)
plt.xlabel('Mean Chi',fontsize=20)
plt.ylabel('Count',fontsize=20)
plt.legend(fontsize=20)

plt.subplot(3,2,3)
plt.grid(True)
plt.hist(np.array(S['_chi_at_boundary']),bins=100,range=(0,1),color='b',alpha=0.5,label="RealData")
plt.tick_params(labelsize=20)
plt.xlabel('Chi at Boundary',fontsize=20)
plt.ylabel('Count',fontsize=20)
plt.legend(fontsize=20)

plt.subplot(3,2,4)
plt.grid(True)
#kaleko: _n_hits_in_largest_cluster_michel => _michel_n_hits
plt.hist(np.array(S['_michel_n_hits']),bins=100,range=(0,100),color='b',alpha=0.5,label="RealData")
plt.tick_params(labelsize=20)
plt.xlabel('N Hits in Michel',fontsize=20)
plt.ylabel('Count',fontsize=20)
plt.legend(fontsize=20)

#kaleko: this used to be _largest_cluster_charge... cosmic_ana doesn't make that variable
plt.subplot(3,2,5)
plt.grid(True)
plt.hist(np.array(S['_michel_clustered_charge'])*0.008*1.4,bins=100,range=(0,200),color='b',alpha=0.5,label="RealData")
plt.tick_params(labelsize=20)
plt.xlabel('Energy [MeV]',fontsize=20)
plt.ylabel('Count/2 MeV',fontsize=20)
plt.legend(fontsize=20)
print "entries = %d, underflow = %d, overflow = %d"%(len(S['_michel_clustered_charge']),S.query('_michel_clustered_charge*0.008*1.4<0').shape[0],S.query('_michel_clustered_charge*0.008*1.4>200').shape[0])
plt.show()


In [ ]:
print "\t!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!"
print "\t!!!!~~~~~~~~~~~~CORRELATIONS~~~~~~~~~~!!!!"
print "\t!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!"
S = REALDATA.query('_michel_clustered_charge > 0')

plt.figure(figsize=(16,32))

plt.subplot(5,2,1)
Q1 = plt.hist2d(np.array(S['_michel_clustered_charge'])*0.008*1.4,
                np.array(S['_lowest_chi']), 
                bins = 100,
                range=np.array([(0, 100), (0,1)]))
plt.colorbar()
plt.tick_params(labelsize=20)
plt.title("RealData",fontsize=20)
plt.xlabel('Energy',fontsize=20)
plt.ylabel('Lowest Chi',fontsize=20)

plt.subplot(5,2,3)
Q3 = plt.hist2d(np.array(S['_michel_clustered_charge'])*0.008*1.4,
                np.array(S['_chi_at_boundary']), 
                bins = 100,
                range=np.array([(0, 100), (0,1)]))
plt.colorbar()
plt.tick_params(labelsize=20)
plt.title("RealData",fontsize=20)
plt.xlabel('Energy',fontsize=20)
plt.ylabel('Chi at Boundary',fontsize=20)

plt.subplot(5,2,5)
Q3 = plt.hist2d(np.array(S['_michel_clustered_charge'])*0.008*1.4,
                np.array(S['_mean_chi']), 
                bins = 100,
                range=np.array([(0, 100), (0,1)]))
plt.colorbar()
plt.tick_params(labelsize=20)
plt.title("RealData",fontsize=20)
plt.xlabel('Energy',fontsize=20)
plt.ylabel('Mean Chi',fontsize=20)

plt.subplot(5,2,7)
Q3 = plt.hist2d(np.array(S['_michel_clustered_charge'])*0.008*1.4,
                np.array(S['_rms_chi']), 
                bins = 100,
                range=np.array([(0, 100), (0,1)]))
plt.colorbar()
plt.tick_params(labelsize=20)
plt.title("RealData",fontsize=20)
plt.xlabel('Energy',fontsize=20)
plt.ylabel('RMS Chi',fontsize=20)

plt.subplot(5,2,9)
Q3 = plt.hist2d(np.array(S['_michel_clustered_charge'])*0.008*1.4,
                np.array(S['_michel_n_hits']), 
                bins = 100,
                range=np.array([(0, 100), (0,100)]))
plt.colorbar()
plt.tick_params(labelsize=20)
plt.title("RealData",fontsize=20)
plt.xlabel('Energy',fontsize=20)
plt.ylabel('N Hits in Michel Cluster',fontsize=20)

plt.show()


In [ ]:
print "\t!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!"
print "\t!!!!~~~~~~~~~~~~WITH CUTS~~~~~~~~~~~~~!!!!"
print "\t!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!"

print 'Applying precuts...'
S = REALDATA.query('_michel_clustered_charge > 0')
print 'Queried charge > 0. Events left=',S.shape[0]
S = S.query('_michel_n_hits >= 10')
print 'Queried n_hits >= 10. Events left=',S.shape[0]
S = S.query('_lowest_chi < 0.28')
print 'Queried lowest chi < 0.28. Events left=',S.shape[0]
S = S.query('_chi_at_boundary < 0.68')
print 'Queried chi at boundary < 0.68. Events left=',S.shape[0]
#S = S.query('_mean_chi > 0.98')
#print 'Queried mean_chi > 0.98. Events left=',S.shape[0]
#temp
S = S.query('_michel_clustered_charge < 8929')
print 'Queried charge < 8929. Events left=',S.shape[0]

plt.figure(figsize=(12,6))
plt.hist(np.array(S['_michel_clustered_charge'])*0.008*1.4,bins=20,range=(0,200),color='b',alpha=0.5,label="Signal")
plt.tick_params(labelsize=20)

plt.xlabel('Energy [MeV]',fontsize=20)
plt.ylabel('Count/2 MeV',fontsize=20)
plt.legend(fontsize=20)
plt.savefig('sample.pdf')
plt.show()


In [ ]:
print "\t!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!"
print "\t!!!!~~~~~~~~~~~~WITH CUTS~~~~~~~~~~~~~!!!!"
print "\t!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!"

plt.figure(figsize=(16,24))

plt.subplot(3,2,1)
plt.hist(np.array(B['lowest_chi']),bins=100,range=(0,1),color='r',alpha=0.5,label="Background")
plt.hist(np.array(S['lowest_chi']),bins=100,range=(0,1),color='b',alpha=0.5,label="Signal")
plt.tick_params(labelsize=20)

plt.xlabel('Lowest Chi',fontsize=20)
plt.ylabel('Count',fontsize=20)
plt.legend(fontsize=20)

plt.subplot(3,2,2)
plt.hist(np.array(B['mean_chi']),bins=100,range=(0,1),color='r',alpha=0.5,label="Background")
plt.hist(np.array(S['mean_chi']),bins=100,range=(0,1),color='b',alpha=0.5,label="Signal")
plt.tick_params(labelsize=20)

plt.xlabel('Mean Chi',fontsize=20)
plt.ylabel('Count',fontsize=20)
plt.legend(fontsize=20)

plt.subplot(3,2,3)
plt.hist(np.array(B['chi_at_boundary']),bins=100,range=(0,1),color='r',alpha=0.5,label="Background")
plt.hist(np.array(S['chi_at_boundary']),bins=100,range=(0,1),color='b',alpha=0.5,label="Signal")
plt.tick_params(labelsize=20)

plt.xlabel('Chi at Boundary',fontsize=20)
plt.ylabel('Count',fontsize=20)

plt.subplot(3,2,4)
plt.hist(np.array(B['_n_hits_in_largest_cluster_michel']),bins=100,range=(0,100),color='r',alpha=0.5,label="Background")
plt.hist(np.array(S['_n_hits_in_largest_cluster_michel']),bins=100,range=(0,100),color='b',alpha=0.5,label="Signal")
plt.tick_params(labelsize=20)

plt.xlabel('N Hits in Michel',fontsize=20)
plt.ylabel('Count',fontsize=20)

plt.subplot(3,2,5)
plt.hist(np.array(B['_number_of_clusters']),bins=100,range=(0,10),color='r',alpha=0.5,label="Background")
plt.hist(np.array(S['_number_of_clusters']),bins=100,range=(0,10),color='b',alpha=0.5,label="Signal")
plt.tick_params(labelsize=20)

plt.xlabel('Number of Clusters',fontsize=20)
plt.ylabel('Count',fontsize=20)

plt.subplot(3,2,6)
plt.hist(np.array(B['_largest_cluster_charge'])*0.008*1.4,bins=100,range=(0,200),color='r',alpha=0.5,label="Background")
plt.hist(np.array(S['_largest_cluster_charge'])*0.008*1.4,bins=100,range=(0,200),color='b',alpha=0.5,label="Signal")
plt.tick_params(labelsize=20)

plt.xlabel('Energy [MeV]',fontsize=20)
plt.ylabel('Count/2 MeV',fontsize=20)
plt.legend(fontsize=20)
plt.show()

In [ ]:
print np.array(S['_largest_cluster_charge']).size
print np.array(SIGNAL['_largest_cluster_charge']).size

In [ ]:
print np.array(B['_largest_cluster_charge']).size
print np.array(BACKGROUND['_largest_cluster_charge']).size

In [ ]:
101.0/53627.0

In [ ]:
S_temp = np.array(S.query('_largest_cluster_charge * 0.008 * 1.4 < 100'))

In [ ]:
S_temp.size